# Import Libraries

In [ ]:
params = {}

In [ ]:
import time
import json
import sys

from data_task_helpers import something

something()

from api_data_task_executioner.data_task_tools import assert_dte_tools_available, get_resolved_parameters_for_connection, initialise_data_task, find_json_arg  # noqa: E402


In [ ]:
   
environment = initialise_data_task("QE Data Task Running", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))
params["sleep_time"] = params.get("sleep_time", 0.2)

if not params:
    environment.log_error("No parameters given!")
    

# Import dependancies

In [ ]:
import jaydebeapi
import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine, text
from jaydebeapi import Connection
import time
from datetime import datetime

# Set Source and Destinaton Database Connections 

In [ ]:
source_environment = get_resolved_parameters_for_connection("EPMS_Source")
destination_environment = get_resolved_parameters_for_connection("EPMS_Destination")

environment.log_message(f'Finished setting source and destination connections')

# Set Filemaker Driver Files

In [ ]:
driver_jar_path = "fmjdbc.jar"
driver_class = "com.filemaker.jdbc.Driver"

# Process File Maker Data

In [ ]:
def open_file_maker_connection():
    try:
        server = source_environment.get("server")
        username = source_environment.get("username")
        password = source_environment.get("password")
        return jaydebeapi.connect(driver_class, server, [username, password], driver_jar_path)
    except Exception as ex:
        return None


def close_file_maker_connection(con: Connection):
    if con is not None:
        con.close()


def fetch_file_maker_data(query: str, con: Connection):
    cursor = con.cursor()
    cursor.execute(query)
    data_rows = cursor.fetchall()
    cursor.close()
    return data_rows

# Open connection to FM

In [ ]:
fm_connection = open_file_maker_connection()

# Create dim_patient Data Frame

In [ ]:
patient_confs = {
    "query": "SELECT  r.name, d.name, f.name, f.code, p.id, p.sex, p.dob, p.maritalStatus, p.patientStatus, p.clientCode, pharmacyNumber, pmtctNumber, nameFirst, nameLast, phoneCellNumber, phoneAltNumber, p.ageYear, resCurrentAddTown, resCurrentAddConstituency, resCurrentAddStreet, resPermanentAddTown, resPermanentAddConstituency, resPermanentAddStreet, idFacilityCurrent, tsName, tsCellPhoneNumber, tsSecondName, tsSecondCellPhoneNumber,  homeBasedCareOrg, homeBasedCareCode,  deathDate  FROM pat p LEFT join fac f on p.idFacilityCurrent = f.id  LEFT join region r on f.idregion=r.id LEFT join district d on f.idDistrict = d.id",
    "cols": ['region','district','current_facility', 'facility_code','client_id', 'sex', 'date_of_birth', 'marital_status','patient_status','client_code', 'pharmacy_code','pmtct_number', 'first_name','last_name','contact_number','alt_contact_number','age','current_town','current_constituency','current_street', 'permanent_town','permanent_constituency','permanent_street','id_facility_current', 'ts_name', 'ts_cell_phone_number', 'ts_second_name', 'ts_second_cell_phone_number', 'cbart_cargs_name', 'cbart_cargs_code', 'death_date']
}

patient_query = patient_confs.get("query")
patient_cols = patient_confs.get("cols")

patient_data = fetch_file_maker_data(patient_query, fm_connection)
dim_pat_df = pd.DataFrame(patient_data, columns=patient_cols)

dim_pat_df['date_of_birth'] = pd.to_datetime(dim_pat_df['date_of_birth'], errors='coerce')
dim_pat_df['first_name'] = 'FName'
dim_pat_df['last_name'] = 'LName'
dim_pat_df['contact_number'] = '123456'
dim_pat_df['alt_contact_number'] = '123456'
dim_pat_df['ts_name'] = 'TS_Name'
dim_pat_df['ts_second_name'] = 'TS_Second_Name'
dim_pat_df['ts_cell_phone_number'] = 'TS_Phone'
dim_pat_df['ts_second_cell_phone_number'] = 'TS_Second_Phone'

dim_pat_df['sex'] = dim_pat_df['sex'].fillna('Unknown')

dim_pat_df.head()

environment.log_message(f'Finished extracting client data')


# Get patient latest transfer status

In [ ]:
fact_tsfr_confs = {
    "query": "SELECT idpatient,  status, \"date\" FROM tsfr",
    "cols": ['client_id' ,'transfer_status', 'transfer_date']
}

fact_tsfr_query = fact_tsfr_confs.get("query")
fact_tsfr_cols = fact_tsfr_confs.get("cols")

fact_tsfr_data = fetch_file_maker_data(fact_tsfr_query, fm_connection)
fact_tsfr_df = pd.DataFrame(fact_tsfr_data, columns=fact_tsfr_cols)

fact_tsfr_df = fact_tsfr_df.dropna(subset=['transfer_date'])

fact_tsfr_df = fact_tsfr_df.loc[fact_tsfr_df.groupby('client_id')['transfer_date'].idxmin()].reset_index(drop=True)
fact_tsfr_df.head()

fact_tsfr_df.head(100)

environment.log_message(f'Finished extracting client transfer status data')

# Add transfer status to patient

In [ ]:
dim_pat_df = pd.merge(dim_pat_df, fact_tsfr_df, on='client_id', how='left')

dim_pat_df.head()

# Get first Weight and WHO Stage

In [ ]:
fact_meas_confs = {
    "query": "SELECT idpatient, weight, whoStage,\"Date\" FROM Meas m",
    "cols": ['client_id', 'weight','who_stage', 'creation_date']
}

fact_meas_query = fact_meas_confs.get("query")
fact_meas_cols = fact_meas_confs.get("cols")

fact_meas_data = fetch_file_maker_data(fact_meas_query, fm_connection)
fact_meas_df = pd.DataFrame(fact_meas_data, columns=fact_meas_cols)

fact_meas_df = fact_meas_df.dropna(subset=['creation_date'])

fact_meas_df = fact_meas_df.loc[fact_meas_df.groupby('client_id')['creation_date'].idxmin()].reset_index(drop=True)
fact_meas_df.head()

# Create fact_hiv_diagnosis Data Frame

In [ ]:
fact_hiv_diagnosis_confs = {
    "query": "SELECT idpatient, hivconfirmationdate, hivconfirmatoryresultsdate, hivconfirmatoryresultstype, idfacilitycreate, fullDisclosureDAte FROM Cd",
    "cols": ['client_id', 'hiv_confirmation_date','hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id','full_disclosure_date']
}

fact_hiv_diagnosis_query = fact_hiv_diagnosis_confs.get("query")
fact_hiv_diagnosis_cols = fact_hiv_diagnosis_confs.get("cols")

fact_hiv_diagnosis_data = fetch_file_maker_data(fact_hiv_diagnosis_query, fm_connection)
fact_hiv_diagnosis_df = pd.DataFrame(fact_hiv_diagnosis_data, columns=fact_hiv_diagnosis_cols)

fact_hiv_diagnosis_df['hiv_confirmation_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmation_date'], errors='coerce')
fact_hiv_diagnosis_df['hiv_confirmatory_result_date'] = pd.to_datetime(fact_hiv_diagnosis_df['hiv_confirmatory_result_date'], errors='coerce')

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.dropna(subset=['hiv_confirmation_date'])

fact_hiv_diagnosis_df = fact_hiv_diagnosis_df.loc[fact_hiv_diagnosis_df.groupby('client_id')['hiv_confirmation_date'].idxmin()].reset_index(drop=True)

fact_hiv_diagnosis_df.head()

environment.log_message(f'Finished extracting HIV diagnosis data')

# Add Initial weight and WHO Stage to fact_hiv_diagnosis

In [ ]:
fact_hiv_diagnosis_df = pd.merge(fact_hiv_diagnosis_df, fact_meas_df[['client_id','weight', 'who_stage']], on='client_id', how='left')

fact_hiv_diagnosis_df.head()

# Add dim_patient, fact_hiv_diagnosis to fact_sentinel_event dataframe

In [ ]:
fact_sentinel_event_df = pd.merge(dim_pat_df, fact_hiv_diagnosis_df, on='client_id', how='left')

fact_sentinel_event_df.head()

# Create hiv_enrolment dataframe 

In [ ]:
fact_hiv_enrolment_confs = {
    "query": "SELECT c.idpatient, c.idfacilitycreate, c.hivenrolleddate, c.disclosureenrollmentdate, c.arteligiblereason, c.artstartdate, p.artnumber, p.artnumber, p.artnumberlegacy, c.idfacilityARTStart FROM pat p LEFT JOIN Cd c ON p.id=c.idpatient",
    "cols": ['client_id', 'hiv_enrollment_facility_id', 'hiv_enrollment_date', 'hiv_disclosure_enrollment_date','art_eligible_reason','art_start_date', 'quantum_number' , 'art_number', 'art_number_legacy', 'arv_initiating_facility']
}


fact_hiv_enrolment_query = fact_hiv_enrolment_confs.get("query")
fact_hiv_enrolment_cols = fact_hiv_enrolment_confs.get("cols")

fact_hiv_enrolment_data = fetch_file_maker_data(fact_hiv_enrolment_query, fm_connection)
fact_hiv_enrolment_df = pd.DataFrame(fact_hiv_enrolment_data, columns=fact_hiv_enrolment_cols)

fact_hiv_enrolment_df['hiv_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_enrollment_date'], errors='coerce')
fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'] = pd.to_datetime(fact_hiv_enrolment_df['hiv_disclosure_enrollment_date'], errors='coerce')
fact_hiv_enrolment_df['art_start_date'] = pd.to_datetime(fact_hiv_enrolment_df['art_start_date'], errors='coerce')

fact_hiv_enrolment_df = fact_hiv_enrolment_df.dropna(subset=['hiv_enrollment_date'])

fact_hiv_enrolment_df = fact_hiv_enrolment_df.loc[fact_hiv_enrolment_df.groupby('client_id')['hiv_enrollment_date'].idxmin()].reset_index(drop=True)

fact_hiv_enrolment_df.head()

environment.log_message(f'Finished extracting HIV enrolment data')

# Add fact_hiv_enrolment to fact_sentinel_event dataframe

In [ ]:
fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_hiv_enrolment_df, on='client_id', how='left')
fact_sentinel_event_df.head()

# Get first and last visit ids per patient

In [ ]:
fact_visits_confs = {
    "query": "SELECT id, idpatient, visitDate FROM Fup",
    "cols": ['id', 'client_id', 'visit_date' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

fact_visits_data = fetch_file_maker_data(fact_visits_query, fm_connection)
fact_visits_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

latest_visits_df = fact_visits_df.loc[fact_visits_df.groupby('client_id')['visit_date'].idxmax()]

first_visits_df = fact_visits_df.loc[fact_visits_df.groupby('client_id')['visit_date'].idxmin()]

# Create first_fact_visits dataframe 

In [ ]:
fact_visits_confs = {
    "query": "SELECT DISTINCT f.idpatient, f.idfacilitycreate, f.visitDate FROM Fup f",
    "cols": ['client_id', 'visit_facility_id', 'visit_date']
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = first_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_first_visit_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_first_visit_df['visit_date'] = pd.to_datetime(fact_first_visit_df['visit_date'], errors='coerce')

fact_first_visit_df.head() 

# Add first visit info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'visit_date': 'first_visit_date', 
    'visit_facility_id': 'first_visit_facility_id'
}

fact_first_visit_df = fact_first_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_visit_df, on='client_id', how='left')
fact_sentinel_event_df.head()

# Create fact_last_visit dataframe 

In [ ]:
fact_visits_confs = {
    "query": "SELECT DISTINCT f.idpatient, f.idfacilitycreate, f.visitDate, f.followupDate, f.scheduledDate, f.careModel, f.pregnantstatus, f.Breastfeeding, f.pregnantlmp, f.pregnantedd, f.ctxadherence, f.arvadherence, cc.treatment, cc.dateTreatment, labResult, pf.stiGenitalUlcers, pf.stiVaginalUrethralDischarge, pf.stiScreenResult, f.oiDetail, f.oiOther, f.tbScreenResult FROM Fup f LEFT JOIN patFup pf on f.id=pf.id LEFT JOIN cc ON f.id=cc.idFollowUp",
    "cols": ['client_id', 'visit_facility_id', 'visit_date', 'next_visit_date', 'scheduled_visit_date', 'care_model', 'pregnancy_status', 'breast_feeding', 'lmp', 'edd', 'ctx_adherence', 'arv_adherence', 'cc_treatment_type','cc_treatment_date','cc_results', 'genital_ulcers','vaginal_urethral_discharge','sti_screening_result', 'oi', 'oi_other','tb_screen_result' ]
}

fact_visits_query = fact_visits_confs.get("query")
fact_visits_cols = fact_visits_confs.get("cols")

ids = latest_visits_df['id'].tolist()

ids_str = ','.join(f"'{id_}'" for id_ in ids)

fact_visits_query_updated = f"{fact_visits_query} WHERE f.id IN ({ids_str})"

fact_visits_data = fetch_file_maker_data(fact_visits_query_updated, fm_connection)
fact_last_visit_df = pd.DataFrame(fact_visits_data, columns=fact_visits_cols)

fact_last_visit_df['visit_date'] = pd.to_datetime(fact_last_visit_df['visit_date'], errors='coerce')
fact_last_visit_df['next_visit_date'] = pd.to_datetime(fact_last_visit_df['next_visit_date'], errors='coerce')

fact_last_visit_df['care_model'] = fact_last_visit_df['care_model'].fillna('Unknown')

fact_last_visit_df.head() 

# Add last visit info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'visit_date': 'last_visit_date',
    'next_visit_date': 'last_next_visit_date', 
    'scheduled_visit_date': 'last_scheduled_visit_date', 
    'pregnancy_status': 'last_pregnancy_status', 
    'breast_feeding': 'last_breast_feeding', 
    'lmp': 'last_lmp', 
    'edd': 'last_edd',
    'care_model': 'last_care_model',
    'visit_facility_id': 'last_visit_facility_id',
    'cc_treatment_type':'last_cc_treatment_type',
    'cc_treatment_date': 'last_cc_treatment_date',
    'cc_results': 'last_cc_results',
    'oi':'last_oi', 
    'oi_other': 'last_oi_other',
    'tb_screen_result': 'last_tb_screen_result'
}

fact_last_visit_df = fact_last_visit_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

last_refresh_date = datetime.strptime(datetime.now().strftime('%Y-%m-%d'), '%Y-%m-%d')

print(f'last_refresh_date: {last_refresh_date}')

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_visit_df, on='client_id', how='left')

fact_sentinel_event_df['last_visit_duration'] = (fact_sentinel_event_df['last_next_visit_date'].dt.year - fact_sentinel_event_df['last_visit_date'].dt.year) * 12 + (fact_sentinel_event_df['last_next_visit_date'].dt.month - fact_sentinel_event_df['last_visit_date'].dt.month)

fact_sentinel_event_df['iit_date'] = fact_sentinel_event_df['last_next_visit_date'] + pd.Timedelta(days=29)

def diff_in_months(date1, date2):
    if pd.isnull(date1) or pd.isnull(date2):
        return None
    return (date1.year - date2.year) * 12 + date1.month - date2.month

fact_sentinel_event_df['months_since_hiv_confirmed'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(last_refresh_date, row['hiv_confirmation_date']), axis=1)

fact_sentinel_event_df['iit_duration'] = (pd.Timestamp('today').normalize() - fact_sentinel_event_df['iit_date']).dt.days

fact_sentinel_event_df['last_PBFW'] = fact_sentinel_event_df.apply(
    lambda row: 'Pregnant' if row['last_pregnancy_status'] == 'Yes' and row['sex'] == 'Female' else 'Breast feeding' if row['last_breast_feeding'] == 'Yes' and row['sex'] == 'Female' else None, 
    axis=1
)


fact_sentinel_event_df['last_PBFW_status'] = fact_sentinel_event_df.apply(
    lambda row: 'Yes' if pd.notna(row['last_PBFW']) and row['sex'] == 'Female' else 'No' if row['sex'] == 'Female' else None,
    axis=1
)

fact_sentinel_event_df.head()

environment.log_message(f'Finished extracting client visits data')

# Create patient regimen dataframe

In [ ]:
fact_reg_confs = {
    "query": "SELECT idpatient, idfacilitycreate, \"date\", type,line, code, dosage,duration, reason FROM rgm",
    "cols": ['client_id', 'ti_facility_id', 'regimen_date', 'regimen_type', 'regimen_line' ,'regimen', 'regimen_dosage','regimen_duration', 'regimen_reason']
}

fact_reg_query = fact_reg_confs.get("query")
fact_reg_cols = fact_reg_confs.get("cols")

fact_reg_data = fetch_file_maker_data(fact_reg_query, fm_connection)
fact_reg_df = pd.DataFrame(fact_reg_data, columns=fact_reg_cols)

fact_reg_df['regimen_date'] = pd.to_datetime(fact_reg_df['regimen_date'], errors='coerce')

fact_reg_df = fact_reg_df.dropna(subset=['client_id', 'regimen_date'])

fact_reg_df = fact_reg_df.loc[fact_reg_df.groupby('client_id')['regimen_date'].idxmax()]

fact_reg_df.head()

environment.log_message(f'Finished extracting client regimen data')

# Add regimen info to fact_sentinel_event

In [ ]:
columns_to_add = ['client_id','regimen_line' ,'regimen','regimen_date']

fact_reg_df = fact_reg_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_reg_df, on='client_id', how='left')

fact_sentinel_event_df = fact_sentinel_event_df.rename(columns={
    'regimen_line': 'last_regimen_line',
    'regimen': 'last_regimen',
    'regimen_date': 'last_regimen_date'
})

fact_sentinel_event_df.head()

# Create fact_ti dataframe 

In [ ]:
fact_ti_confs = {
    "query": "SELECT idpatient, idfacilitycreate, interruptionDate,interruptionReason, interruptionReasonOther, restartDate,duration FROM ti",
    "cols": ['client_id', 'ti_facility_id', 'art_interruption_date', 'art_interruption_reason', 'art_interruption_reason_other' ,'art_restart_date', 'interruption_duration']
}

fact_ti_query = fact_ti_confs.get("query")
fact_ti_cols = fact_ti_confs.get("cols")

fact_ti_data = fetch_file_maker_data(fact_ti_query, fm_connection)
fact_ti_df = pd.DataFrame(fact_ti_data, columns=fact_ti_cols)

fact_ti_df['art_restart_date'] = pd.to_datetime(fact_ti_df['art_restart_date'], errors='coerce')

fact_ti_df = fact_ti_df.dropna(subset=['client_id', 'art_interruption_date'])

fact_ti_df = fact_ti_df.loc[fact_ti_df.groupby('client_id')['art_interruption_date'].idxmax()]

fact_ti_df.head()

environment.log_message(f'Finished extracting client treatment interruption data')

# Add treatment interription info to fact_sentinel_event

In [ ]:
columns_to_add = ['client_id', 'art_interruption_date', 'art_interruption_reason', 'art_interruption_reason_other' ,'art_restart_date']

fact_ti_df = fact_ti_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_ti_df, on='client_id', how='left')
fact_sentinel_event_df.head()

# Create fact_tbt dataframe 

In [ ]:
fact_tbt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, category, startDate, regimen, duration, stopDateExpected, stopDateActual , registrationNumber , site, siteDetail , CASE  WHEN outcome = '1' THEN 'Cured' WHEN outcome = '2' THEN 'Treatment complete' WHEN outcome = '3' THEN 'Died' WHEN outcome = '4' THEN 'Failure' WHEN outcome = '5' THEN 'Lost to follow-up' WHEN outcome = '6' THEN 'Not evaluated' END AS outcome, devCreationTimestamp  FROM tbt",
    "cols": ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_regimen', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome', 'tbt_outcome_date']
}

fact_tbt_query = fact_tbt_confs.get("query")
fact_tbt_cols = fact_tbt_confs.get("cols")

fact_tbt_data = fetch_file_maker_data(fact_tbt_query, fm_connection)
fact_tbt_df = pd.DataFrame(fact_tbt_data, columns=fact_tbt_cols)

fact_tbt_df['tbt_outcome_date'] = pd.to_datetime(fact_tbt_df['tbt_outcome_date'])
fact_tbt_df['tbt_start_date'] = pd.to_datetime(fact_tbt_df['tbt_start_date'])
fact_tbt_df['tbt_outcome_date'] = fact_tbt_df['tbt_outcome_date'].dt.date

fact_tbt_df.loc[fact_tbt_df['tbt_outcome'].isnull(), 'tbt_outcome_date'] = None

fact_tbt_df = fact_tbt_df.dropna(subset=['tbt_start_date'])

fact_tbt_df = fact_tbt_df.loc[fact_tbt_df.groupby('client_id')['tbt_start_date'].idxmax()]

fact_tbt_df.head()

environment.log_message(f'Finished extracting client tbt data')

# Add tbt info to fact_sentinel_event

In [ ]:
columns_to_add = ['client_id', 'tbt_facility_id', 'tbt_category', 'tbt_start_date', 'tbt_regimen', 'tbt_duration' ,'tbt_expected_stop_date', 'tbt_actual_stop_date', 'tbt_registration_number', 'tbt_site', 'tbt_site_detail', 'tbt_outcome', 'tbt_outcome_date']

fact_tbt_df = fact_tbt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tbt_df, on='client_id', how='left')
fact_sentinel_event_df.head()


# Create fact_tpt dataframe 

In [ ]:
fact_tpt_confs = {
    "query": "SELECT idpatient, idfacilitycreate, iptRegimen, startDate, duration, stopDateExpected, stopDateActual , CASE  WHEN stopReason = '1' THEN 'Completed TPT' WHEN stopReason = '2' THEN 'Cough' WHEN stopReason = '3' THEN 'Confirmed TB' WHEN stopReason = '4' THEN 'Hepatitis' WHEN stopReason = '5' THEN 'Neuropathy' WHEN stopReason = '6' THEN 'Poor Adherence' WHEN stopReason = '7' THEN 'Medicine Out Of Stock' WHEN stopReason = '8' THEN stopReasonOther ELSE '' END AS stopReason, status, adherenceDetail FROM ipt",
    "cols": ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_adherance']
}

fact_tpt_query = fact_tpt_confs.get("query")
fact_tpt_cols = fact_tpt_confs.get("cols")

fact_tpt_data = fetch_file_maker_data(fact_tpt_query, fm_connection)
fact_tpt_df = pd.DataFrame(fact_tpt_data, columns=fact_tpt_cols)

fact_tpt_df = fact_tpt_df.dropna(subset=['client_id', 'tpt_start_date'])

fact_tpt_df = fact_tpt_df.loc[fact_tpt_df.groupby('client_id')['tpt_start_date'].idxmax()]

fact_tpt_df['tpt_start_date'] = pd.to_datetime(fact_tpt_df['tpt_start_date'], errors='coerce')
fact_tpt_df['tpt_actual_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_actual_stop_date'], errors='coerce')
fact_tpt_df['tpt_expected_stop_date'] = pd.to_datetime(fact_tpt_df['tpt_expected_stop_date'], errors='coerce')

fact_tpt_df['tpt_duration_two'] = fact_tpt_df.apply(
    lambda row: (row['tpt_expected_stop_date'] - row['tpt_start_date']).days 
                if pd.notna(row['tpt_expected_stop_date']) and pd.notna(row['tpt_start_date']) 
                else None,
    axis=1
)

fact_tpt_df.loc[
    (fact_tpt_df['tpt_regimen'] == '3H') & (fact_tpt_df['tpt_start_date'] > '2020-08-01'),
    'tpt_regimen'
] = '3HP'

fact_tpt_df.head()

environment.log_message(f'Finished extracting client tpt data')

In [ ]:
# Check for invalid dates in the 'tpt_start_date' column
invalid_dates = fact_tpt_df[pd.to_datetime(fact_tpt_df['tpt_expected_stop_date'], errors='coerce').isna()]

# Display rows with invalid dates
invalid_dates.head()

# Add tpt info to fact_sentinel_event

In [ ]:
columns_to_add = ['client_id', 'tpt_facility_id', 'tpt_regimen', 'tpt_start_date', 'tpt_duration', 'tpt_duration_two' ,'tpt_expected_stop_date', 'tpt_actual_stop_date', 'tpt_stop_reason', 'tpt_status', 'tpt_adherance']

fact_tpt_df = fact_tpt_df[columns_to_add]

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_tpt_df, on='client_id', how='left')

fact_sentinel_event_df.head()

# Compute TPT Type

In [ ]:
fact_sentinel_event_df['tpt_type'] = fact_sentinel_event_df.apply(
    lambda row: "9H" if pd.notna(row['tpt_start_date']) and row['tpt_duration_two'] >= 250 else
                ("6H" if pd.notna(row['tpt_start_date']) and row['tpt_duration_two'] >= 168 else
                 ("3HP" if pd.notna(row['tpt_start_date']) and row['tpt_start_date'] >= pd.to_datetime("2020-08-01") and 
                          pd.notna(row['tpt_expected_stop_date']) and row['tpt_duration_two'] >= 77 else
                  ("3HP" if row['tpt_regimen'] in ["3H", "3HP"] and row['tpt_start_date'] >= pd.to_datetime("2020-08-01") else None))),
    axis=1
)

fact_sentinel_event_df['tpt_type'] = fact_sentinel_event_df.apply(
    lambda row: "3HP" if pd.isna(row['tpt_type']) and 
                          row['tpt_regimen'] in ["3H", "3HP"] and 
                          row['tpt_start_date'] >= pd.to_datetime("2020-08-01") else row['tpt_type'],
    axis=1
)

fact_sentinel_event_df.head()

# Compute tpt_status_two_outcome

In [ ]:
def compute_tpt_status_two_outcome(row):
    # Define constants
    tpt_complete_other = 146
    tpt_complete_3HP = 70
    tpt_on_tx_3H = 113
    tpt_on_tx_other = 240

    # Calculate the time differences for various conditions
    start_to_actual_stop = (row['tpt_actual_stop_date'] - row['tpt_start_date']).days if pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) else None
    start_to_last_refresh = (last_refresh_date - row['tpt_start_date']).days if pd.notna(row['tpt_start_date']) else None

    if pd.isna(row['tpt_start_date']):
        return "No TPT Documentation"
    elif pd.notna(row['tpt_start_date']) and pd.notna(row['tpt_actual_stop_date']) and row['tpt_start_date'] >= row['tpt_actual_stop_date']:
        return "Wrong TPT Dates"
    elif pd.isna(row['tpt_start_date']) and pd.notna(row['tpt_actual_stop_date']):
        return "Wrong TPT Dates"
    elif pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_regimen']) and pd.isna(row['tpt_expected_stop_date']) and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is blank"
    elif pd.notna(row['tpt_start_date']) and row['tpt_expected_stop_date'] == row['tpt_start_date'] and pd.isna(row['tpt_regimen']) and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is blank"
    elif pd.notna(row['tpt_start_date']) and row['tpt_expected_stop_date'] == row['tpt_start_date'] and pd.notna(row['tpt_regimen']) and row['tpt_regimen'] == "Other" and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is other"
    elif pd.isna(row['tpt_type']) and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop >= tpt_complete_other:
        return "6-month TPT Completed but TPT Type is unknown"
    elif pd.isna(row['tpt_type']) and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_complete_other:
        return "6-month TPT Completed but TPT Type is unknown"
    elif row['tpt_status'] == "Active" and row['tpt_type'] == "3HP" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh < tpt_on_tx_3H:
     return "Still on 3HP"
    elif row['tpt_status'] == "Active" and row['tpt_type'] == "6H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh < tpt_on_tx_other:
        return "Still on 6H"
    elif row['tpt_status'] == "Active" and row['tpt_type'] == "9H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh < tpt_on_tx_other:
        return "Still on 9H"
    elif row['tpt_type'] == "3HP" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_on_tx_3H:
     return "3HP Completed but TPT Stop Date is blank"
    elif row['tpt_type'] == "3HP" and start_to_actual_stop >= tpt_complete_3HP:
     return "3HP Completed"
    elif row['tpt_type'] == "3HP" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_3HP:
     return "3HP Stopped Before Completion"
    elif row['tpt_type'] == "6H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop >= tpt_complete_other:
        return "6H Completed"
    elif row['tpt_type'] == "6H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_on_tx_other:
        return "6H Completed but TPT Stop Date is blank"
    elif row['tpt_type'] == "6H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_other:
        return "6H stopped Before 6-month Completion"
    elif row['tpt_type'] == "9H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop >= tpt_complete_other:
        return "6-month of 9H Completed"
    elif row['tpt_type'] == "9H" and pd.isna(row['tpt_actual_stop_date']) and start_to_last_refresh >= tpt_on_tx_other:
        return "6-months of 9H Completed but TPT Stop Date is blank"
    elif row['tpt_type'] == "9H" and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_other:
        return "9H stopped Before 6-month Completion"
    elif pd.isna(row['tpt_type']) and pd.notna(row['tpt_actual_stop_date']) and start_to_actual_stop < tpt_complete_other:
        return "TPT stopped before 6-month completion & TPT Type is unknown"
    elif pd.notna(row['tpt_start_date']) and row['tpt_start_date'] == row['tpt_expected_stop_date'] and pd.isna(row['tpt_actual_stop_date']):
        return "TPT regimen is blank"
    else:
        return None


fact_sentinel_event_df['tpt_status_two_outcome'] = fact_sentinel_event_df.apply(compute_tpt_status_two_outcome, axis=1)

fact_sentinel_event_df.head()

# Create fact_lab dataframe 

In [ ]:
fact_lab_confs = {
    "query": "SELECT l.idpatient, l.idfacilitycreate, l.orderDate, l.result, l.resultDate, lm.name, l.status FROM lab l INNER JOIN LabMaster lm ON lm.id=l.idtest",
    "cols": ['client_id', 'lab_facility_id', 'order_date', 'lab_result', 'result_date', 'test_name','status' ]
}

fact_lab_query = fact_lab_confs.get("query")
fact_lab_cols = fact_lab_confs.get("cols")

fact_lab_data = fetch_file_maker_data(fact_lab_query, fm_connection)
fact_lab_df = pd.DataFrame(fact_lab_data, columns=fact_lab_cols)

fact_lab_df['order_date'] = pd.to_datetime(fact_lab_df['order_date'], errors='coerce')
fact_lab_df['result_date'] = pd.to_datetime(fact_lab_df['result_date'], errors='coerce')


fact_lab_df.head()

environment.log_message(f'Finished extracting client lab data')

# Create fact_first_viral_load dataframe

In [ ]:
fact_first_viral_load_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_first_viral_load_tests_df = fact_first_viral_load_tests_df.sort_values('order_date').groupby('client_id').first().reset_index()

def categorize_result(result):
    if result < 40:
        return 'Undetectable'
    elif 40 <= result < 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_first_viral_load_tests_df['category'] = fact_first_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_first_viral_load_tests_df.head()

# Create fact_last_viral_load dataframe

In [ ]:
fact_last_viral_load_tests_df = fact_lab_df[fact_lab_df['test_name'] == 'Viral Load']
fact_last_viral_load_tests_df = fact_last_viral_load_tests_df.sort_values('order_date').groupby('client_id').last().reset_index()

def categorize_result(result):
    if result < 40:
        return 'Undetectable'
    elif 40 <= result < 1000:
        return 'LLV'
    else:
        return 'Unsuppressed'

fact_last_viral_load_tests_df['category'] = fact_last_viral_load_tests_df['lab_result'].apply(categorize_result)

fact_last_viral_load_tests_df.head()

# Add first viral load info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'order_date': 'first_viral_load_order_date',
    'result_date': 'first_viral_load_result_date', 
    'lab_result': 'first_viral_load_result',
    'lab_facility_id': 'first_viral_load_facility_id',
    'category': 'first_viral_load_result_category',
    'status': 'first_viral_load_result_status',
}

fact_first_viral_load_tests_df = fact_first_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_first_viral_load_tests_df, on='client_id', how='left')

fact_sentinel_event_df.head()

# Add last viral load info to fact_sentinel_event

In [ ]:
columns_to_add = {
    'order_date': 'last_viral_load_order_date',
    'result_date': 'last_viral_load_result_date', 
    'lab_result': 'last_viral_load_result',
    'lab_facility_id': 'last_viral_load_facility_id',
    'category': 'last_viral_load_result_category',
    'status': 'last_viral_load_result_status',
}

fact_last_viral_load_tests_df = fact_last_viral_load_tests_df[list(columns_to_add.keys()) + ['client_id']].rename(columns=columns_to_add)

fact_sentinel_event_df = pd.merge(fact_sentinel_event_df, fact_last_viral_load_tests_df, on='client_id', how='left')

# Add duration of ART in months

In [ ]:
def diff_in_months(date1, date2):
    if pd.isnull(date1) or pd.isnull(date2):
        return None
    return (date1.year - date2.year) * 12 + date1.month - date2.month

fact_sentinel_event_df['months_since_art_start'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(last_refresh_date, row['art_start_date']), axis=1)

fact_sentinel_event_df['months_since_art_restart'] = fact_sentinel_event_df.apply(lambda row: diff_in_months(last_refresh_date, row['art_restart_date']), axis=1)

fact_sentinel_event_df.head()

# Add program status

In [ ]:
def classify_status(row):
    if (pd.notna(row['patient_status']) and
        pd.notna(row['art_start_date']) and
        row['patient_status'] == 'Active ART' and
        (last_refresh_date - pd.to_datetime(row['art_start_date'])).days <= 31 and
        (pd.isna(row['last_next_visit_date']) or pd.isna(row['last_visit_date']))):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and
          pd.isna(row['last_next_visit_date']) and
          (last_refresh_date - pd.to_datetime(row['last_visit_date'])).days <= 365 and
          row['patient_status'] == 'Active ART'):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and row['patient_status'] == 'Deceased'):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and row['patient_status'] == 'ART Stopped'):
        return row['patient_status']
    
    elif (pd.notna(row['patient_status']) and row['patient_status'] == 'Transferred Out'):
        return row['patient_status']
    
    elif (pd.notna(row['iit_date']) and
          pd.to_datetime(row['iit_date']) > last_refresh_date and
          (pd.notna(row['art_start_date']) or pd.notna(row['last_regimen']))):
        return 'Active ART'
    
    elif (pd.notna(row['iit_date']) and
          pd.to_datetime(row['iit_date']) <= last_refresh_date and
          (pd.notna(row['art_start_date']) or pd.notna(row['last_regimen']))):
        return 'IIT'
    
    else:
        return None
    
fact_sentinel_event_df['client_status'] = fact_sentinel_event_df.apply(classify_status, axis=1)

fact_sentinel_event_df['tx_curr'] = fact_sentinel_event_df['client_status'].apply(
    lambda status: 'Yes' if status == 'Active ART' else 'No'
)

fact_sentinel_event_df.head()

# Add Upto date on TPT

In [ ]:
# Define constants
tpt_complete_other = 146
tpt_complete_3HP = 70

# Ensure the calculation happens for each row using apply
fact_sentinel_event_df['up_to_date_tpt'] = fact_sentinel_event_df.apply(
    lambda row: 1 if (
        # Check if tx_curr is 'Yes'
        row['tx_curr'] == 'Yes' and (
            # Condition 1: TPT completed for 3HP
            (pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) and
             (row['tpt_actual_stop_date'] - row['tpt_start_date']).days >= tpt_complete_3HP and
             row['tpt_type'] in ["3HP", "3H"]) or
            # Condition 2: TPT completed for other regimens
            (pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) and
             (row['tpt_actual_stop_date'] - row['tpt_start_date']).days >= tpt_complete_other and
             row['tpt_type'] in ["6HP", "6H", "9H", "9HP", None])  # None for not listed regimens
        )
    ) else 0,
    axis=1
)
fact_sentinel_event_df.head()

# Add currently on TPT

In [ ]:
# Define constants
tpt_on_tx_3H = 113
tpt_on_tx_other = 240

# Update the DataFrame
fact_sentinel_event_df['currently_on_tpt'] = fact_sentinel_event_df.apply(
    lambda row: 1 if (
        row['tx_curr'] == 'Yes' and (
            # Condition 1: TPT initiated but not completed for 3HP or 3H
            (pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']) and
             (last_refresh_date - row['tpt_start_date']).days < tpt_on_tx_3H and
             row['tpt_type'] in ["3HP", "3H"]) or
            # Condition 2: TPT initiated but not completed for 6H
            (pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']) and
             (last_refresh_date - row['tpt_start_date']).days < tpt_on_tx_other and
             row['tpt_type'] in ["6H","6HP"]) or
            # Condition 3: Currently on TB treatment (less than 1 year since TB treatment start)
            (pd.notna(row['tbt_start_date']) and
             (last_refresh_date - row['tbt_start_date']).days < 365.25 and 
             pd.isna(row['tbt_actual_stop_date']))
        )
    ) else 0,
    axis=1
)

# Display the updated DataFrame
fact_sentinel_event_df.head()


# Add Incomplete or Never Initiated TPT

In [ ]:
# Define constants
tpt_on_tx_3H = 112
tpt_on_tx_other = 239

tpt_incomplete_tx_3H = 70
tpt_incomplete_tx_other = 146

# Calculate 'incomplete_or_never_initiated_tpt' column
fact_sentinel_event_df['incomplete_or_never_initiated_tpt'] = fact_sentinel_event_df.apply(
    lambda row: 1 if (
        row['tx_curr'] == 'Yes' and (
            # Condition 1: Initiation date is filled, but actual stop date is missing
            (
                pd.notna(row['tpt_start_date']) and pd.isna(row['tpt_actual_stop_date']) and (
                    # >112 days for 3HP
                    ((last_refresh_date - row['tpt_start_date']).days > tpt_on_tx_3H and
                     row['tpt_type'] in ["3HP", "3H"]) or
                    # >239 days for other regimens
                    ((last_refresh_date - row['tpt_start_date']).days > tpt_on_tx_other and
                     (row['tpt_type'] in ["6HP", "6H", "9H", "9HP"] or pd.isna(row['tpt_type'])))
                )
            ) or
            # Condition 2: Actual stop date is filled, but duration is less than required
            (
                pd.notna(row['tpt_actual_stop_date']) and pd.notna(row['tpt_start_date']) and (
                    # <70 days for 3HP
                    ((row['tpt_actual_stop_date'] - row['tpt_start_date']).days < tpt_incomplete_tx_3H and
                     row['tpt_type'] in ["3HP", "3H"]) or
                    # <146 days for other regimens
                    ((row['tpt_actual_stop_date'] - row['tpt_start_date']).days < tpt_incomplete_tx_other and
                     (row['tpt_type'] in ["6HP", "6H", "9H", "9HP"] or pd.isna(row['tpt_type'])))
                )
            ) or
            # Condition 3: No initiation date is filled, and not on TB treatment
            (
                pd.isna(row['tpt_start_date']) and (pd.isna(row['tbt_start_date']) or 
                (pd.notna(row['tbt_start_date']) and pd.notna(row['tbt_actual_stop_date'])))
            ) or 
            # Condition 4: No TPT initiation date is filled, started on TB treatment, no completion date but been on TBT for more than 1 year
            (
                pd.isna(row['tpt_start_date']) and (pd.notna(row['tbt_start_date']) and pd.isna(row['tbt_actual_stop_date'])
                and (last_refresh_date - row['tbt_start_date']).days > 365.25)
            )
        )
    ) else 0,
    axis=1
)

fact_sentinel_event_df.head()

# Compute tpt_status_two

In [ ]:
def calculate_tpt_two_status(row):
    if row['up_to_date_tpt'] == 1:
        return 'TPT completed'
    elif row['currently_on_tpt'] == 1:
        return 'Currently on TPT/TBT'
    elif row['incomplete_or_never_initiated_tpt'] == 1:
        return 'Incomplete TPT'

fact_sentinel_event_df['tpt_status_two'] = fact_sentinel_event_df.apply(calculate_tpt_two_status, axis=1)

fact_sentinel_event_df.head()

# Add Retention for patients

In [ ]:
def retention_status(row):
    days_missed = (pd.to_datetime(last_refresh_date) - pd.to_datetime(row['last_next_visit_date'])).days if pd.notna(row['last_next_visit_date']) else None
    
    if row['tx_curr'] == 'Yes' and pd.notna(row['last_regimen']):
        if pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] >= last_refresh_date:
            return 'In Care'
        elif pd.isna(row['last_next_visit_date']):
            return 'In Care'
        elif pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] < last_refresh_date and 1 <= days_missed <= 28:
            return 'Missed Appointments'

    if row['tx_curr'] == 'No' and pd.isna(row['art_start_date']):
        if pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] >= last_refresh_date:
            return 'In Care'
        elif pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] < last_refresh_date and 1 <= days_missed <= 28:
            return 'Missed Appointments'
    
    if pd.notna(row['last_next_visit_date']) and row['last_next_visit_date'] < last_refresh_date and 29 <= days_missed <= 90:
        return 'Treatment Interruptions'
    
    return None

fact_sentinel_event_df['retention_status'] = fact_sentinel_event_df.apply(retention_status, axis=1)

# Add patient ages at different stages cascade

In [ ]:
fact_sentinel_event_df['hiv_confirmation_age'] = (fact_sentinel_event_df['hiv_confirmation_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['hiv_enrollment_age'] = (fact_sentinel_event_df['hiv_enrollment_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_viral_load_result_age'] = (fact_sentinel_event_df['first_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_viral_load_result_age'] = (fact_sentinel_event_df['last_viral_load_result_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['first_visit_age'] = (fact_sentinel_event_df['first_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['last_visit_age'] = (fact_sentinel_event_df['last_visit_date'] - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25
fact_sentinel_event_df['current_age'] = (last_refresh_date - fact_sentinel_event_df['date_of_birth']).dt.days // 365.25

fact_sentinel_event_df.head()

# Add ART Duration

In [ ]:
fact_sentinel_event_df['art_duration'] = (
    (last_refresh_date.year - fact_sentinel_event_df['art_start_date'].dt.year) * 12 +
    (last_refresh_date.month - fact_sentinel_event_df['art_start_date'].dt.month)
)

fact_sentinel_event_df['art_duration'] = (last_refresh_date - fact_sentinel_event_df['art_start_date']).dt.days // 365.25

fact_sentinel_event_df.head()

# Days since last visit

In [ ]:
fact_sentinel_event_df['days_since_last_visit'] = (last_refresh_date - fact_sentinel_event_df['last_visit_date']).dt.days
fact_sentinel_event_df['years_since_last_visit'] = (last_refresh_date - fact_sentinel_event_df['last_visit_date']).dt.days // 365.25

fact_sentinel_event_df.head()

# Create dim age groups

In [ ]:
bins = [0, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, float('inf')]
labels = ['0-4', '5-9', '10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64', '65+']

dim_age_group_df = pd.DataFrame({
    'age': range(0, 101)
})

dim_age_group_df['pepfar_age_group'] = pd.cut(dim_age_group_df['age'], bins=bins, labels=labels, right=False)

dim_age_group_df['paeds_adult_age_group'] = dim_age_group_df['age'].apply(lambda x: 'Pediatric' if x <= 19 else 'Adult')

def tri_pillar_18_classify(age):
    if 0 <= age <= 18:
        return '0-18'
    elif age == 19:
        return '19'
    elif 20 <= age <= 39:
        return '20-39'
    else:
        return '40+'

def tri_pillar_classify(age):
    if 0 <= age <= 19:
        return '0-19'
    elif 20 <= age <= 39:
        return '20-39'
    else:
        return '40+'
    
dim_age_group_df['tri_pillar_age_group_eighteen'] = dim_age_group_df['age'].apply(tri_pillar_18_classify)

dim_age_group_df['tri_pillar_age_group'] = dim_age_group_df['age'].apply(tri_pillar_classify)

dim_age_group_df.head(100)

# Add age group sort column

In [ ]:
def tri_pillar_age_group_val(age):
    if 0 <= age <= 19:
        return 1
    elif 20 <= age <= 39:
        return 2
    else:
        return 3

dim_age_group_df['tri_pillar_age_group_val'] = dim_age_group_df['age'].apply(tri_pillar_age_group_val)

dim_age_group_df['paeds_adult_age_group_val'] = dim_age_group_df['paeds_adult_age_group'].apply(
    lambda x: 1 if x == '0-18 yr old (Pediatric)' else 2
)

def pepfar_age_group_val(label):
    if label == '0-4':
        return 1
    elif label == '5-9':
        return 2
    elif label == '10-14':
        return 3
    elif label == '15-19':
        return 4
    elif label == '20-24':
        return 5
    elif label == '25-29':
        return 6
    elif label == '30-34':
        return 7
    elif label == '35-39':
        return 8
    elif label == '40-44':
        return 9
    elif label == '45-49':
        return 10
    elif label == '50-54':
        return 11
    elif label == '55-59':
        return 12
    elif label == '60-64':
        return 13
    else:  # '65+'
        return 14

dim_age_group_df['pepfar_age_group_val'] = dim_age_group_df['pepfar_age_group'].apply(pepfar_age_group_val)

dim_age_group_df.head(100)

# Create a column that contains current age groups

In [ ]:
fact_sentinel_event_df = fact_sentinel_event_df.merge(
    dim_age_group_df[['age','pepfar_age_group', 'pepfar_age_group_val','paeds_adult_age_group', 'paeds_adult_age_group_val','tri_pillar_age_group','tri_pillar_age_group_val']], 
    left_on='current_age', 
    right_on='age', 
    how='left'
)

fact_sentinel_event_df.head()

# Calculate the difference in months between last_refresh_date and last_viral_load_result_date

In [ ]:
def calculate_months_since_last_viral_load(date):
    return (last_refresh_date.year - date.year) * 12 + last_refresh_date.month - date.month

fact_sentinel_event_df['months_since_last_viral_load'] = fact_sentinel_event_df['last_viral_load_result_date'].apply(calculate_months_since_last_viral_load)
fact_sentinel_event_df['months_since_last_viral_load_order'] = fact_sentinel_event_df['last_viral_load_order_date'].apply(calculate_months_since_last_viral_load)

fact_sentinel_event_df['days_since_last_viral_load_order'] = (last_refresh_date - fact_sentinel_event_df['last_viral_load_order_date']).dt.days


fact_sentinel_event_df.head()

# Add VL Eligibility

In [ ]:
def classify_vl_eligibility(row):
    # Monitored per Guidelines
    if row['tx_curr'] == 'Yes':
        if row['months_since_art_start'] < 6 or row['months_since_art_restart'] < 6 :
            return 'Monitored per Guidelines'

        if row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'

        if (row['last_breast_feeding'] == 'Yes' or row['last_pregnancy_status'] == 'Yes') and row['months_since_last_viral_load'] <= 3:
            return 'Monitored per Guidelines'

        if row['current_age'] > 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 12:
            return 'Monitored per Guidelines'

        if row['current_age'] <= 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] <= 6:
            return 'Monitored per Guidelines'

    # Slightly Delayed
    if row['tx_curr'] == 'Yes':
        if row['last_viral_load_result'] >= 40 and 3 < row['months_since_last_viral_load'] <= 4:
            return 'Slightly Delayed'

        if (row['last_breast_feeding'] == 'Yes' or row['last_pregnancy_status'] == 'Yes') and 3 < row['months_since_last_viral_load'] <= 4:
            return 'Slightly Delayed'

        if row['current_age'] > 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and 12 < row['months_since_last_viral_load'] <= 14:
            return 'Slightly Delayed'

        if row['current_age'] <= 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and 6 < row['months_since_last_viral_load'] <= 8:
            return 'Slightly Delayed'
                
        # add patients with an order date and no result and the test was done in the last 14 days
        if row['days_since_last_viral_load_order'] <= 14 and pd.isna(row['last_viral_load_result']):
            return 'Slightly Delayed'

    # Delayed
    if row['tx_curr'] == 'Yes':
        if row['last_viral_load_result'] >= 40 and row['months_since_last_viral_load'] > 4:
            return 'Delayed'

        if (row['last_breast_feeding'] == 'Yes' or row['last_pregnancy_status'] == 'Yes') and row['months_since_last_viral_load'] > 4:
            return 'Delayed'

        if row['current_age'] > 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 14:
            return 'Delayed'

        if row['current_age'] <= 19 and row['last_breast_feeding'] != 'Yes' and row['last_pregnancy_status'] != 'Yes' and row['last_viral_load_result'] < 40 and row['months_since_last_viral_load'] > 8:
            return 'Delayed'

    # Default to Delayed if TxCurr is 'Yes' and no other conditions are met
    if row['tx_curr'] == 'Yes':
        return 'Delayed'

    # If none of the conditions are met, return 'Not Classified'
    return None


fact_sentinel_event_df['vl_eligibility'] = fact_sentinel_event_df.apply(lambda row: classify_vl_eligibility(row), axis=1)

fact_sentinel_event_df.head()

# Add reason for next VL

In [ ]:
def categorize_vl_reason(row):
    if row['months_since_art_start'] is not None and row['months_since_art_start'] > 6 and pd.notnull(row['art_start_date']) and pd.isnull(row['first_viral_load_result_date']):
        return 'First Test'
    elif row['months_since_last_viral_load'] > 3 and row['last_pregnancy_status'] == 'Yes':
        return 'Pregnant Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes':
        return 'BF Women'
    elif row['months_since_last_viral_load'] > 3 and row['last_viral_load_result_category'] == 'LLV':
        return 'Low Level Viraemia'
    elif row['months_since_last_viral_load'] > 6 and row['current_age'] <= 19:
        return 'Children And Adolescents Under 19'
    elif row['months_since_last_viral_load'] > 12 and row['current_age'] > 19:
        return 'Treatment Monitoring For Adults'
    else:
        return None

fact_sentinel_event_df['reason_for_next_vl'] = fact_sentinel_event_df.apply(categorize_vl_reason, axis=1)

fact_sentinel_event_df.head()

# Add next VL date

In [ ]:
def calculate_next_vl_date(row):
    if pd.isnull(row['last_viral_load_result_date']):
        return None
    elif row['reason_for_next_vl'] == 'First Test':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Pregnant Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'BF Women':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=3)
    elif row['reason_for_next_vl'] == 'Low Level Viraemia':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Children And Adolescents Under 19':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=6)
    elif row['reason_for_next_vl'] == 'Treatment Monitoring For Adults':
        return row['last_viral_load_result_date'] + pd.DateOffset(months=12)
    else:
        return None

fact_sentinel_event_df['next_vl_date'] = fact_sentinel_event_df.apply(calculate_next_vl_date, axis=1)


# Compute if patient has ever been initiated on ART  

In [ ]:
def determine_has_been_initiated_on_art(row):
    return 'Yes' if pd.notna(row['art_start_date']) else 'No'
    
fact_sentinel_event_df['has_ever_been_initiated_on_art'] = fact_sentinel_event_df.apply(lambda row: determine_has_been_initiated_on_art(row), axis=1)

fact_sentinel_event_df.head()

# Check if last VL is valid

In [ ]:
def is_last_viral_load_valid(row, df):
    row = df.iloc[row.name]

    if pd.notna(row['last_viral_load_result_date']) and pd.notna(row['art_restart_date']):
        if pd.to_datetime(row['last_viral_load_result_date']) < pd.to_datetime(row['art_restart_date']):
            return 'No'
        
    if (row['tx_curr'] == 'Yes' and
        row['last_PBFW_status'] == 'Yes' and
        pd.notna(row['last_viral_load_result']) and
        pd.notna(row['art_start_date']) and
        (last_refresh_date - pd.to_datetime(row['art_start_date'])).days >= 90 and
        row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          (row['last_PBFW_status'] == 'No' or pd.isna(row['last_PBFW_status'])) and
          row['current_age'] >= 20 and
          pd.notna(row['last_viral_load_result']) and
          pd.notna(row['art_start_date']) and
          (last_refresh_date - pd.to_datetime(row['art_start_date'])).days >= 182 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'

    elif (row['tx_curr'] == 'Yes' and
          (row['last_PBFW_status'] == 'No' or pd.isna(row['last_PBFW_status'])) and
          row['current_age'] <= 19 and
          pd.notna(row['last_viral_load_result']) and
          pd.notna(row['art_start_date']) and
          (last_refresh_date - pd.to_datetime(row['art_start_date'])).days >= 182 and
          row['vl_eligibility'] == 'Monitored per Guidelines'):
        return 'Yes'
        
    else:
        return 'No'
    
fact_sentinel_event_df['is_last_viral_load_valid'] = fact_sentinel_event_df.apply(is_last_viral_load_valid, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Check if patient is virally suppressed

In [ ]:
def determine_is_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Undetectable':
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Check if patient is not virally suppressed

In [ ]:
def determine_is_not_virally_suppressed(row, df):
    row = df.iloc[row.name]
    if row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result_category'] == 'Unsuppressed':
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['is_not_virally_suppressed'] = fact_sentinel_event_df.apply(determine_is_not_virally_suppressed, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Compute needs VL test

In [ ]:
def determine_needs_VL_test(row, df):
    row = df.iloc[row.name]
    if (row['months_since_art_start'] > 6 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Undetectable' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 12 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] != 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['months_since_art_start'] > 3 and pd.notna(row['first_viral_load_result_date']) and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Undetectable' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 6 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    elif (row['last_viral_load_result_category'] == 'Unsuppressed' and row['last_next_visit_date'] >= last_refresh_date and row['months_since_last_viral_load'] > 3 and row['last_breast_feeding'] == 'Yes') and (row['is_not_virally_suppressed'] != 'Yes' and row['is_virally_suppressed'] != 'Yes'):
        return 'Yes'
    else:
        return None
    
fact_sentinel_event_df['needs_vl_test'] = fact_sentinel_event_df.apply(determine_needs_VL_test, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Assign MMP Status

In [ ]:
def assign_mmp(row,df):
    row = df.iloc[row.name]
    if (row['tx_curr'] == 'Yes' and 
        pd.notna(row['last_next_visit_date']) and 
        pd.notna(row['last_visit_date']) and
        0 <= (pd.to_datetime(row['last_next_visit_date']) - pd.to_datetime(row['last_visit_date'])).days < 84):
        return '< 3 MMP'
    elif (row['tx_curr'] == 'Yes' and
          pd.notna(row['last_next_visit_date']) and 
          pd.notna(row['last_visit_date']) and
          84 <= (pd.to_datetime(row['last_next_visit_date']) - pd.to_datetime(row['last_visit_date'])).days < 168):
        return '3-5 MMP'
    elif (row['tx_curr'] == 'Yes' and
          pd.notna(row['last_next_visit_date']) and
          pd.notna(row['last_visit_date']) and
          (pd.to_datetime(row['last_next_visit_date']) - pd.to_datetime(row['last_visit_date'])).days >= 168):
        return '6+ MMP'
    elif (row['tx_curr'] == 'Yes' and
          pd.isna(row['last_next_visit_date'])):
        return '6+ MMP'
    else:
        return None

fact_sentinel_event_df['mmp_status'] = fact_sentinel_event_df.apply(assign_mmp, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()


# Add Six MMP Eligibility

In [ ]:
def determine_six_mmp_eligibility(row, df):
    row = df.iloc[row.name]
    if (row['current_age'] > 2 and (pd.isnull(row['tbt_start_date']) or row['tbt_duration'] >= 6) and (row['is_last_viral_load_valid'] == 'Yes' and row['last_viral_load_result'] < 40) and row['months_since_art_start'] >= 6  and (row['last_regimen_line'] == '1' or row['last_regimen_line'] == '2') and pd.isnull(row['last_oi_other'])):
        return 'Yes'
    else:
        return None
        
max_last_visit_month = last_refresh_date.month
max_last_visit_year = last_refresh_date.year

if max_last_visit_month == 1:  # Handle the edge case for January
    prev_last_visit_month = 12
    prev_last_visit_year = max_last_visit_year - 1
else:
    prev_last_visit_month = max_last_visit_month - 1
    prev_last_visit_year = max_last_visit_year

fact_sentinel_event_df['last_visit_month'] = fact_sentinel_event_df['last_visit_date'].dt.month
fact_sentinel_event_df['last_visit_year'] = fact_sentinel_event_df['last_visit_date'].dt.year

fact_sentinel_event_df['mmp_status_current'] = fact_sentinel_event_df.apply(
    lambda row: row['mmp_status'] if (row['last_visit_month'] == prev_last_visit_month and row['last_visit_year'] == prev_last_visit_year) else None,
    axis=1
)

fact_sentinel_event_df['mmp_status_adult'] = fact_sentinel_event_df['paeds_adult_age_group'].apply(lambda x: 'Yes' if 'adult' in str(x).lower() else None)

fact_sentinel_event_df['mmp_status_paed'] = fact_sentinel_event_df['paeds_adult_age_group'].apply(lambda x: 'Yes' if 'pediatric' in str(x).lower() else None)

fact_sentinel_event_df['six_mmp_eligible'] = fact_sentinel_event_df.apply(determine_six_mmp_eligibility, axis=1, df=fact_sentinel_event_df)

fact_sentinel_event_df.head()

# Add Six MMP Eligibility But Not Given

In [ ]:
fact_sentinel_event_df['six_mmp_eligible_but_not_given'] = fact_sentinel_event_df.apply(
    lambda row: 'Yes' if row['six_mmp_eligible'] == 'Yes' and row['mmp_status'] != '6+ MMP' else 'No', 
    axis=1
)

fact_sentinel_event_df.head()

# Compute currrent cascade status

In [ ]:
def determine_art_outcomes(row):
    if row['is_virally_suppressed'] == 'Yes':
        return 'VL Suppressed'
    elif row['is_not_virally_suppressed'] == 'Yes':
        return 'VL Not Suppressed'
    elif row['needs_vl_test'] == 'Yes':
        return 'Needs VL Test'
    elif row['months_since_art_start'] < 6:
        return 'Recently initiated'
    elif row['has_ever_been_initiated_on_art'] == 'No':
        return 'Not initiated on ART'
    elif row['years_since_last_visit'] > 2:
        return 'Case Closed'
    elif row['patient_status'] == 'Deceased':
        return 'Died'
    else:
        return 'Unknown'

fact_sentinel_event_df['art_outcomes'] = fact_sentinel_event_df.apply(determine_art_outcomes, axis=1)

fact_sentinel_event_df.head()

environment.log_message(f'Finished adding computed columns')

# Change dates from datetime to date 

In [ ]:
fact_sentinel_event_df = fact_sentinel_event_df.apply(lambda col: col.dt.date if col.dtype == 'datetime64[ns]' else col)

fact_sentinel_event_df.head()

# Reorder the columns in fact_sentinel_event_df

In [ ]:
new_column_order = [
    'client_id', 'region', 'district', 'facility_code', 'current_facility', 'id_facility_current',  'transfer_status', 'transfer_date',
    'quantum_number', 'art_number', 'art_number_legacy', 'client_code', 'pharmacy_code', 
    'pmtct_number',  'first_name', 'last_name', 'contact_number', 'alt_contact_number', 'date_of_birth', 'sex', 'marital_status',  'current_age',  'pepfar_age_group', 'tri_pillar_age_group', 'paeds_adult_age_group', 
    'ts_name', 'ts_cell_phone_number', 'ts_second_name', 'ts_second_cell_phone_number',
    'weight','current_town', 'current_constituency', 'current_street', 'permanent_town', 'permanent_constituency', 'permanent_street', 
    'cbart_cargs_name', 'cbart_cargs_code','hiv_confirmation_date', 'hiv_confirmation_age', 'hiv_confirmatory_result_date', 'hiv_confirmatory_result_type', 'hiv_diagnosis_facility_id',
    'hiv_enrollment_date', 'hiv_enrollment_age','hiv_disclosure_enrollment_date', 'full_disclosure_date', 'hiv_enrollment_facility_id','arv_initiating_facility','art_start_date', 'months_since_art_start', 'months_since_art_restart','art_duration', 'has_ever_been_initiated_on_art', 
    'patient_status', 'who_stage', 'art_eligible_reason',  'first_visit_date', 'first_visit_age', 'first_visit_facility_id', 'last_visit_date', 'last_visit_age',  'last_next_visit_date', 'last_care_model', 'last_visit_facility_id', 
    'last_cc_treatment_type', 'last_cc_treatment_date', 'last_cc_results', 'last_oi', 'last_scheduled_visit_date', 'last_pregnancy_status', 'last_breast_feeding', 
    'last_oi_other', 'last_regimen_line', 'last_regimen', 'last_regimen_date', 'last_visit_duration', 'days_since_last_visit', 'years_since_last_visit', 'last_visit_month', 'last_visit_year',   'last_lmp', 'last_edd','last_tb_screen_result', 'first_viral_load_order_date', 'first_viral_load_result_date', 'first_viral_load_result_age',  
    'first_viral_load_result', 'first_viral_load_facility_id', 'first_viral_load_result_category', 'first_viral_load_result_status', 'last_viral_load_order_date', 
    'last_viral_load_result_date','last_viral_load_result_age', 'months_since_last_viral_load', 'last_viral_load_result', 'last_viral_load_facility_id', 'is_virally_suppressed', 'is_not_virally_suppressed', 'needs_vl_test', 'last_viral_load_result_status', 'reason_for_next_vl', 'next_vl_date', 
    'art_interruption_date', 'art_interruption_reason', 'art_interruption_reason_other' ,'art_restart_date',
    'tbt_start_date','tbt_regimen', 'tbt_expected_stop_date', 'tbt_actual_stop_date','tbt_facility_id', 'tbt_category',  'tbt_duration', 'tbt_registration_number', 
    'tbt_site', 'tbt_site_detail', 'tbt_outcome', 'tbt_outcome_date', 'tpt_facility_id', 'tpt_start_date', 'tpt_regimen', 'tpt_expected_stop_date', 'tpt_status','tpt_status_two_outcome','tpt_type',
    'tpt_actual_stop_date', 'tpt_stop_reason',  'tpt_adherance','death_date', 'iit_date', 'iit_duration', 'client_status', 'retention_status', 'tx_curr', 
    'is_last_viral_load_valid', 'last_viral_load_result_category',  'vl_eligibility', 'tpt_duration', 'tpt_duration_two',  'tpt_status_two', 'last_PBFW', 'last_PBFW_status','mmp_status', 'mmp_status_current', 'mmp_status_adult', 'mmp_status_paed', 'six_mmp_eligible', 'six_mmp_eligible_but_not_given', 'art_outcomes'    
]

# Reorder the columns in the DataFrame
fact_sentinel_event_df = fact_sentinel_event_df[new_column_order]

# Display the reordered DataFrame
fact_sentinel_event_df.head()

# Clean up column names

In [ ]:
def rename_columns(col):
    if '_id' in col:
        return '__' + col
    else:
        col = col.replace('_', ' ').title()
        col = col.replace('Tpt', 'TPT')
        col = col.replace('Tbt', 'TBT')
        col = col.replace('Hiv', 'HIV')
        col = col.replace('Art', 'ART')
        col = col.replace('Iit', 'IIT')
        col = col.replace('Mmp', 'MMP')
        col = col.replace('Pmtct', 'PMTCT')
        col = col.replace('Pbfw', 'PBFW')
        col = col.replace('Vl', 'Viral Load')
        col = col.replace('Tx Curr', 'TX Curr')
        col = col.replace('Oi', 'OI')
        col = col.replace('Lmp', 'LMP')
        col = col.replace('Edd', 'EDD')
        col = col.replace('Who', 'WHO')
        col = col.replace('Tb', 'TB')
        return col

fact_sentinel_event_df.columns = [rename_columns(col) for col in fact_sentinel_event_df.columns]

fact_sentinel_event_df.head()

# Define Destination Database Connection

In [ ]:
# PostgreSQL server connection details
server = destination_environment.get("server")
database = destination_environment.get("database")
port = destination_environment.get("port")
username = destination_environment.get("username")
password = destination_environment.get("password")

# Create connection to the default database (e.g., postgres) to check if the target database exists
default_database = 'postgres'  # Usually, 'postgres' is used for administrative tasks
default_connection_url = f'postgresql://{username}:{password}@{server}:{port}/{default_database}'

# Set the isolation level to AUTOCOMMIT for creating the database
default_engine = create_engine(default_connection_url, isolation_level='AUTOCOMMIT')

# Query to check if the database exists
check_db_query = f"SELECT 1 FROM pg_database WHERE datname = '{database}'"

# Query to create the database if it doesn't exist
create_db_query = f"CREATE DATABASE {database}"

try:
    with default_engine.connect() as connection:
        # Check if the database exists
        result = connection.execute(text(check_db_query)).fetchone()

        if not result:
            # If the database does not exist, create it
            connection.execute(text(create_db_query))
            environment.log_message(f"Database '{database}' created successfully.")
        else:
            environment.log_message(f"Database '{database}' already exists.")
except Exception as e:
    print(f"Error occurred: {e}")

# Now connect to the target database
connection_url = f'postgresql://{username}:{password}@{server}:{port}/{database}'
engine = create_engine(connection_url)

# Function that checks if a table exists

In [ ]:


def create_and_insert_table(df, table_name, engine, schema_name):
    """
    This function checks if a schema and table exist in the given schema. If the schema does not exist,
    it creates the schema. If the table does not exist, it creates the table and inserts data from the
    provided DataFrame. If the table exists, it replaces the data.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing data to be inserted.
    table_name (str): The name of the table to create or replace.
    engine (sqlalchemy.engine.Engine): The SQLAlchemy engine to connect to the PostgreSQL database.
    schema_name (str): The schema name where the table is located. Default is 'public'.
    """
    
    # SQL query to check if the schema exists
    check_schema_query = f"""
    SELECT EXISTS (
        SELECT 1 FROM information_schema.schemata 
        WHERE schema_name = '{schema_name}'
    );
    """
    
    # SQL query to check if the table exists
    check_table_query = f"""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_schema = '{schema_name}' 
        AND table_name = '{table_name}'
    );
    """
    
    try:
        with engine.connect() as connection:
            # Begin a new transaction
            with connection.begin() as transaction:
                # Check if the schema exists
                schema_exists = connection.execute(text(check_schema_query)).fetchone()

                if not schema_exists[0]:  # If the schema doesn't exist
                    environment.log_message(f"Schema '{schema_name}' does not exist. Creating it...")
                    # Create the schema
                    connection.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_name};"))
                    environment.log_message(f"Schema '{schema_name}' created successfully.")

            # Check if the table exists (in a separate transaction to avoid conflicts)
            with connection.begin() as transaction:
                table_exists = connection.execute(text(check_table_query)).fetchone()

                if not table_exists[0]:  # If the table doesn't exist
                    environment.log_message(f"Table '{table_name}' does not exist in schema '{schema_name}'. Creating it...")
                    # Automatically create the table based on the DataFrame structure
                    df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema=schema_name)
                    environment.log_message(f"Table '{table_name}' created and data inserted successfully.")
                else:
                    # If the table exists, replace the data
                    df.to_sql(table_name, con=engine, if_exists='replace', index=False, schema=schema_name)
                    environment.log_message(f"Data replaced successfully in table '{table_name}'.")

    except Exception as e:
        print(f"Error occurred: {e}")




# Write dim age group to table

In [ ]:
table_name = 'dim_age_group'
schema_name = 'analysis'

create_and_insert_table(dim_age_group_df, table_name, engine, schema_name)

dim_age_group_df.head()

# Drop View

In [ ]:
drop_views_sql = '''
DROP VIEW IF EXISTS "analysis"."score_card";
'''

try:
    with engine.connect() as conn:
        with conn.begin():
            conn.execute(text(drop_views_sql))
            print("View dropped successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

# Remove NULL Characters

In [ ]:
for col in problematic_columns:
    fact_sentinel_event_df[col] = fact_sentinel_event_df[col].str.replace('\x00', '', regex=False)

# Write sentinel event to table

In [ ]:
table_name = 'fact_sentinel_event'
schema_name = 'analysis'

create_and_insert_table(fact_sentinel_event_df, table_name, engine, schema_name)

fact_sentinel_event_df.head()

# Get current facility

In [ ]:
facility_counts = dim_pat_df['current_facility'].value_counts()
current_facility = facility_counts.idxmax()

# Add last refresh date

In [ ]:
data = {
    'last_refresh_date': [datetime.today().strftime('%Y-%m-%d')],
    'facility_name': current_facility
}

last_refresh_date_df = pd.DataFrame(data)

print(last_refresh_date_df)

# Write last refresh to table 

In [ ]:
table_name = 'dim_last_refresh'
schema_name = 'analysis'


create_and_insert_table(last_refresh_date_df, table_name, engine, schema_name)

last_refresh_date_df.head()

# Create Score Card View

In [ ]:
view_score_card_creation_query = """
CREATE VIEW analysis.score_card AS
 WITH ordered_data AS (
         SELECT '6+ MMP'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."MMP Status" = '6+ MMP'::text THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            1 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'TPT'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."TPT Status Two" = 'TPT completed'::text THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            2 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'Retention'::text AS indicator,
            sum(
                CASE
                    WHEN (fact_sentinel_event."Retention Status" = ANY (ARRAY['In Care'::text, 'Missed Appointments'::text])) THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."Retention Status" = ANY (ARRAY['In Care'::text, 'Missed Appointments'::text, 'Treatment Interruptions'::text]) THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            3 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'VLM'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND (fact_sentinel_event."Viral Load Eligibility" = ANY (ARRAY['Monitored per Guidelines'::text, 'Slightly Delayed'::text])) THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            4 AS order_col
           FROM analysis.fact_sentinel_event
        UNION ALL
         SELECT 'VLS (VL<40 CPS/ML)'::text AS indicator,
            sum(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."Is Virally Suppressed" = 'Yes'::text THEN 1
                    ELSE 0
                END)::numeric * 100.0 / count(
                CASE
                    WHEN fact_sentinel_event."TX Curr" = 'Yes'::text AND fact_sentinel_event."Is Last Viral Load Valid" = 'Yes'::text THEN 1
                    ELSE NULL::integer
                END)::numeric AS score,
            5 AS order_col
           FROM analysis.fact_sentinel_event
        )
 SELECT indicator,
    round(score, 2) AS score,
        CASE
            WHEN indicator = '6+ MMP'::text AND score < 50::numeric THEN 'Bad'::text
            WHEN indicator = '6+ MMP'::text AND score >= 50::numeric AND score < 75::numeric THEN 'Average'::text
            WHEN indicator = '6+ MMP'::text AND score >= 75::numeric THEN 'Good'::text
            WHEN indicator = 'TPT'::text AND score < 85::numeric THEN 'Bad'::text
            WHEN indicator = 'TPT'::text AND score >= 85::numeric AND score < 95::numeric THEN 'Average'::text
            WHEN indicator = 'TPT'::text AND score >= 95::numeric THEN 'Good'::text
            WHEN indicator = 'Retention'::text AND score < 90::numeric THEN 'Bad'::text
            WHEN indicator = 'Retention'::text AND score >= 90::numeric AND score < 95::numeric THEN 'Average'::text
            WHEN indicator = 'Retention'::text AND score >= 95::numeric THEN 'Good'::text
            WHEN indicator = 'VLM'::text AND score < 80::numeric THEN 'Bad'::text
            WHEN indicator = 'VLM'::text AND score >= 80::numeric AND score < 90::numeric THEN 'Average'::text
            WHEN indicator = 'VLM'::text AND score >= 90::numeric THEN 'Good'::text
            WHEN indicator = 'VLS (VL<40 CPS/ML)'::text AND score < 85::numeric THEN 'Bad'::text
            WHEN indicator = 'VLS (VL<40 CPS/ML)'::text AND score >= 85::numeric AND score < 93::numeric THEN 'Average'::text
            WHEN indicator = 'VLS (VL<40 CPS/ML)'::text AND score >= 93::numeric THEN 'Good'::text
            ELSE 'Unknown'::text
        END AS color
   FROM ordered_data
  ORDER BY order_col;
"""

try:
    with engine.connect() as conn:
        with conn.begin():
            conn.execute(text(view_score_card_creation_query))
            environment.log_message("Score Card view created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

# Drop dataframes

In [ ]:
del dim_pat_df
del fact_hiv_diagnosis_df
del fact_hiv_enrolment_df
del fact_first_visit_df
del fact_last_visit_df
del fact_visits_df
del fact_lab_df
del fact_tpt_df
del fact_meas_df
del fact_first_viral_load_tests_df
del fact_last_viral_load_tests_df
del fact_tbt_df
del fact_tsfr_df

environment.log_message(f'Finished writing data to database')

# Log end time

In [ ]:
pd.Timestamp.today()